In [ ]:
# conda install beautifulsoup4,                             installしたら消す
# conda install requests
# conda install selenium
# conda install Scrapy

In [ ]:
# 基本

In [ ]:
import urllib.request

url = "https://~"
savefile = "sample.png"

urllib.request.urlretrieve(url, savefile)
print("保存しました")

# 一度pythonのメモリにデータを取得してから、ファイルに保存する。
mem = urllib.request.urlopen(url).read()
with open(savefile, mode="wb") as f:
    f.write(mem)
    print("保存しました")

In [ ]:
# 文字列を変換
url = "https://~"
res = urllib.request.urlopen(url)
data = res.read()

test = data.decode("utf-8")
print(data)

In [ ]:
# 住所検索
import urllib.request as req
import urllib.parse as parse

API = "https://api.aoikujira.com/zip/xml/get.php"

values = {'fml' : 'xml',  #'xml' or 'ini' or 'json'
          'zn' : '1500042'} # 調べるもの、一般化のsys.argv[]はここ！
params = parse.urlencode(values)

url = API + "?" + params
print("url = ", url)

data = req.urlopen(url).read()
text = data.decode("utf-8")
print(text)

# 上３行の別の記述
with req.urlopen(url) as r:
    b = r.read()
    data = b.decode('utf-8')
    print(data)
    
# コマンドラインで一般化もできる

In [ ]:
# BeautifulSoup

In [ ]:
# idで検索
from bs4 import BeautifulSoup

html = """
<html><body>
    <h1 id="title">スクレイピングとは</h1>
    <p id="body">Webページから任意のデータを抽出すること</p>
</body></html>
"""

soup = BeautifulSoup(html, 'html.parser')

title = soup.find(id = "title")
body = soup.find(id = "body")

print("#title =" + title.string)
print("#body =" + body.string)

In [ ]:
# 複数のタグを一気に入手する
html = """
<html><body>
  <ul>
    <li><a href="https://~~">uta</a></li>
    <li><a href="https://--">oto</a></li>
  </ul>
</body></html>
"""

soup = BeautifulSoup(html, 'html.parser')

links = soup.find_all("a")

for a in links:
    href = a.attrs['href']
    text = a.string
    print(text, ">", href)

# uta > https://~~    oto > https://--   と出力される。

In [ ]:
# urlからデータを抽出
from bs4 import BeautifulSoup
import urllib.request as req

url = "https://api.aoikujira.com/zip/xml/1500042"
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

ken = soup.find("ken").string
shi = soup.find("shi").string 
cho = soup.find("cho").string
print(ken, shi, cho)

# urlopen()でURLを開いて、BeautifulSoupで抽出する

In [ ]:
# file ---> index.html
"""
<html><body>
  <div id="main-goods" role="page">
    <h1>フルーツや野菜</h1>
    <ul id="fr-list">
      <li class="red green" data-lo="jp">リンゴ</li>
      <li class="purple" data-lo="us">ブドウ</li>
      <li class="yellow" data-lo="us">レモン</li>
      <li class="yellow" data-lo="jp">オレンジ</li>
    </ul>
    <ul id="ve-list">
      <li class="white green" data-lo="jp">ダイコン</li>
      <li class="red green" data-lo="us">パプリカ</li>
      <li class="black" data-lo="jp">ナス</li>
      <li class="black" data-lo="us">アボカド</li>
      <li class="white" data-lo="cn">レンコン</li>
    </ul>
  </div>
</body></html>
"""

In [ ]:
from bs4 import BeautifulSoup
fp = open("index.html", encoding="utf-8")
soup = BeautifulSoup(fp, "html.parser")

#アボカドを抜き出す
#CSSセレクター
print(soup.select_one("li:nth-of-type(8)").string)
print(soup.select_one("#ve-list > li:nth-of-type(4)").string)
print(soup.select("#ve-list > li[data-lo="us"]")[1].string)
print(soup.select("#ve-list > li.black")[1].string)
#findメソッド
cond = {"data-lo":"us", "class":"black"}
print(soup.find("li", cond).string)
print(soup.find(id="ve-list").find("li", cond).string)

In [ ]:
#正規表現
import re

html = """
<ul>
  <li><a href="hoge.html">hoge</li>
  <li><a href="https://exaple.com/fuga">fuga*</li>
  <li><a href="https://exaple.com/foo">foo*</li>
  <li><a href="http://exaple.com/aaa">aaa</li>
</ul>
"""
soup = BeautifulSoup(html, "html.parser")
li = soup.find_all(href=re.compile(r"^https://"))
for e in li: print(e.attrs['href'])

In [ ]:
# 相対パスから絶対パス
from urllib.parse import urljoin

base = "http://example.com/html/a.html"

print( urljoin(base, "b.html") )                       #--->   http://example.com/html/b.html
print( urljoin(base, "sub/c.html") )                   #--->   http://example.com/html/sub/c.html
print( urljoin(base, "../img/hoge.png") )              #--->   http://example.com/index.html
print( urljoin(base, "/hoge.html") )                   #--->   http://example.com/hoge.html
print( urljoin(base, "http://kujirahand.com/wiki") )   #--->   http://kujirahand.com/wiki
print( urljoin(base, "//uta.pw/shodou") )              #--->   http://uta.pw/shodou

In [2]:
# 丸ごとダウンロードプログラム
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from  os import makedirs
import os.path, time, re

proc_files = {}

# HTML内にあるリンクを抽出する関数
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']")   #CSS
    links += soup.select("a[href]")  #リンク
    result = []
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# ファイルをダウンロードする関数
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath):
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    if os.path.exists(savepath): return savepath
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1)
        return savepath
    except:
        print("ダウンロード失敗：", url)
        return None

# HTMLを解析してダウンロードする関数
def analize_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: return
    proc_files[savepath] = True
    print("analize_html=", url)
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    for link_url in links:
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue
        if re.search(r".(html|htm)$", link_url):
            analize_html(link_url, root_url)
            continue
        download_file(link_url)
        
if __name__ == "__main__":
    url = "https://docs.python.jp/3.6/library/"
    analize_html(url, url)

In [ ]:
# request

In [ ]:
# requests モジュール
r = requests.get("https://~")   # get
formdata = {"username" : "yuuki", 
            "password" : "1234567890"}
d = requests.post("https://~", data=formdata) # post   cf. session(), put("URL"), delete("URL"), head("URL")

print(r.text)
print(r.content)

In [ ]:
# selenium  (今回は firefox を使う)

In [ ]:
# まずはドライバーが必要
# firefox をインストールしたら、https://github.com/mozilla/geckodriver/releases で検索してドライバーをダウンロード
# 日本語対応させたい場合は、調べる

from selenium.webdriver import Firefox, FirefoxOptions
# ヘッドレスモード
options = FirefoxOptions()
options.add_argument('-headless')
browser = Firefox(options=options)
# ページにアクセス
url = "https://~"
browser.get(url)
# (選択)スクショ保存をする
browser.save_screenshot("scsho.png")
browser.quit()
# (選択)テキストボックスに文字を入力 idはディベロッパーツールで調べる
e = browser.find_element_by_id("user")
e.clear()
e.send_keys("yuuki")
e = browser.find_element_by_id("password")
e.clear()
e.send_keys("0123456789")
frm = browser.find_element_by_css_selector("#loginForm form")
frm.submit()   # フォームを送信
# ページのロードまで待つのでsleep(10)くらいを追加しておく
# class名 .islogin
a = browser.find_element_by_css_selector(".islogin a")
url_mypage = a.get_attribute("href")  # mypage のURL
# マイページ表示
browser.get(url_my_page)

In [ ]:
# 定期的な情報収集  macOS, Linux   -->  Ubuntu を使う
# dataget.pyの中
import urllib.request as req
import datetime
import json
API = "URL"
json_str = req.urlopen(API).read().decode("utf-8")   # .textとかで試してみる
data = json.loads(json_str)
print("情報を取得しました")
t = datetime,date.today()
fname = t.strfname("%Y-%m-%d") + ".json"
with open(fname, "w", encoding="utf-8") as f:
    f.write(json_str)
    
# nano エディターをインストール
# cron をつかうことで　上記のファイルを定期実行することができ、自動でデータを取得することができる。

In [ ]:
# Scrapy

In [ ]:
# プロジェクトの作成
scrapy startproject soseki_list  # シェルに打つ
"""
以下のようなファイル群が生成
soseki_list
  scrapy.cfg   --- 設定ファイル
  soseki_list  --- この下がPythonモジュール
    __init__.py
    items.py   --- アイテム定義
    middlewares.py   --- ミドルウェア
    pipelines.py   --- パイプライン
    settings.py  --- 設定
    spiders   --- spider を配置するディレクトリー
      __init__.py
"""

In [ ]:
# soseki_list/spiders/soseki.py
import scrapy

class SosekiSpider(scrapy.Spider):
    name = 'soseki'   # spider の名前
    start_urls = ['https://www.aozora.gr.jp/index_pages/person148.html']  # 取得対象のウェブサイト
    
    def parse(self, response):
        title = response.css('title')
        print(title.extract())    # リスト型で取り出す
        
# scrapy crawl soseki --nolog  を実行

In [ ]:
# soseki_list/spiders/soseki2.py
import scrapy

class Soseki2Spider(scrapy.Spider):
    name = 'soseki2'
    start_urls = ['https://www.aozora.gr.jp/index_pages/person148.html']
    
    def parse(self, response):
        li_list = response.css('ol > li a')
        for a in li_list:
            href = a.css('::attr(href)').extract_first()
            text = a.css('::text').extract_first()
            href2 = response.urljoin(href)  #  相対パスから絶対パスへの変換
            yield{'text' : 'text', 
                   'url' : 'href2'}   # 結果はreturn でなくyieldで返す。

# scrapy crawl soseki2 -o list.json
# python3
# >>> import json, pprint
# >>> a = json.load(open("list.json"))
# >>> pprit.pprint(a)

# Scrapy をシェルで実行
scrapy shell
[1] fetch('https://~~')
[2] response.css('title::text').extract_first()
[3] response.css('#tblIndex > tr td:nth-child(2) a::text').extract()
[4] quit()

In [ ]:
# 特定データをダウンロードする
# scrapy genspider soseki3 www.aozora.gr.jp

# soseki_list/spiders/soseki3.py
import scrapy, pprint
class Soseki4Spider(scrapy.Spider):
    name = 'soseki4'
    allowed_domains = ['www.aozora.gr.jp']
    start_urls = ['htts://ww.aozora.gr.jp/index_pages/person148.html']
    
    def parse(self, response):
        li_list = response.css('ol > li a')
        for a in li_list:
            href = a.css('::attr(href)').extract_first()
            href2 = response.urljoin(href)
            yield response.follow(href2, self.parse_card)
            
    def parse_card(self, response):
        title = response.css('title::text').extract_first()
        alist = response.css('table.download tr td a')
        for a in alist:
            href = a.css('::attr(href)').extract_first()
            href2 = response.urljoin(href)
            if href2[-4:] != ".zip": continue
            req = scrapy.Request(href2, callback=self.parse_item)  #ダウンロードの指示
            req.meta["title"] = title
            yield req
            
    def parse_item(self, response):
        title = response.meta["title"]
        title = title.replace('図書カード：', '').strip()
        fname = title + ".zip"
        with open(fname, "wb") as f:
            f.write(response.body)
            
# scrapy crawl soseki4

In [ ]:
# scrapy のミドルウェアに selenium を組み込む(Javascriptなどを読み込めるようになる)

In [ ]:
# scrapy startproject sakusibbs
# sakusibbs/selenium_middleware.py
from scarapy.http import HtmlResponse
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

driver = Firefox()

def selenium_get(url): # 任意のURLを開く
    driver.get(url)
    
def get_dom(query): # 読み込み完了まで待機
    dom = WedDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, query)))
    return dom

def selenium_close():
    driver.close()

class SeleniumMiddleware(object):  # ミドルウェア本体
    def process_request(self, request, spider): # リクエストをseleniumで処理する
        driver.get(request.url)
        return HtmlResponse(driver.current_url, 
                             body = driver.page_source,
                             encoding = 'utf-8',
                             request = request)

In [ ]:
# scrapy genspider getall uta.pw
# sakusibbs/spiders/getall.py
import scrapy, pprint
from ..selenium_middleware import *

USER = "yuuki"
PASS = "123456789"

class GetallSpider(scrapy.Spider):
    name = 'getall'
    #ミドルウェアを登録
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES":{
            "sakusibbs.selenium_middleware.SeleniumMiddleware" : 0
        }
    }
    
    #ログイン
    def start_requests(self):
        url = 'https://uta.pw/sakusibbs/users.php?action=login'
        selenium_get(url)  #ページを開く
        user = get_dom('#user') # 入力
        user.send_keys(USER) 
        pw = get_dom('#pass')
        pw.send_keys(PASS)
        btn = get_dom('#loginForm input[type=submit]')  # ボタンをクリック
        btn.click()
        a = get_dom('.islogin a')  # ユーザーページを得る
        mypage = a.get_getattribute('href')
        print("mypage:", mypage)
        yield scrapy.Request(mypage, self.parse)
        
    def parse(self, response):
        for a in list:
            url = a.css('::attr(href)').extract_first()
            url2 = response.urljoin(url)
            yield response.follow(url2, self.parse_sakuhin)
            
    def parse_sakuhin(self, response):
        title = response.css('title::text').extract_first()
        print("---", title)
        src = response.css('iframe::attr(src)').extract_first()
        src2 = response.urljoin(src)
        req = scrapy.Request(src2, self.parse_download)
        req.meta["title"] = title
        yield req
        
    def parse_download(self, response):
        title = response.meta["title"]
        fname = title + ".html"
        with open(fname, "wt") as f:
            f.write(response.body)
            
    def closed(self, reason):
        selenium_close()

In [ ]:
# Web API
# http://Webservice.rakuten.co.jp/document/
# http://developer.yahoo.co.jp/Webapi/shopping/
# オープンデータAPI
# http://www.e-stat.go.jp/api/  政府統計の総合窓口
# http://iss.ndl.go.jp/information/api/  国会国立図書館
# https://www.hellowork.go.jp/info/online02.html  ハローワーク
# http://www.land.mlit.go.jp/Webland/api.html  土地総合情報システム

In [ ]:
# 株価、為替情報のスクレイピング    -->   調べる(スクレイピングが禁止されていないか)

In [ ]:
# 抽出は、開発者ツール(ディベロッパーツール)を使うか、ソースコードの表示を用いる